# Scoras Academy

Bem-vindos!

## Contexto

Bem vindo ao maior curso de agentes de IA da Lingua Portuguesa! 
Há muita empolgação em torno da construção de agentes porque eles podem automatizar uma ampla gama de tarefas que antes eram impossíveis.

Na prática, porém, é incrivelmente difícil construir sistemas que executem essas tarefas de forma confiável. À medida em que colocamos agentes em produção, aprendemos que frequentemente é necessário mais controle. Você pode precisar que um agente sempre chame uma ferramenta específica primeiro ou use prompts diferentes com base em seu estado.

Para enfrentar esse problema, usamos o [LangGraph](https://langchain-ai.github.io/langgraph/) — um framework para construir aplicações de agentes e multi-agentes. Separado do pacote LangChain, a  ideia central de design do LangGraph é ajudar os desenvolvedores a adicionar melhor precisão e controle aos fluxos de trabalho dos agentes, adequados à complexidade dos sistemas do mundo real.

## Estrutura do Curso

O curso é estruturado como um conjunto de módulos, com cada módulo focado em um tema particular relacionado ao LangGraph. Você verá uma pasta para cada módulo, que contém uma série de notebooks. Um vídeo acompanhará cada notebook para ajudar a explicar os conceitos, mas os notebooks também são independentes, o que significa que contêm explicações e podem ser visualizados independentemente dos vídeos. 


Antes de começar, por favor, siga as instruções no `README` para criar um ambiente e instalar as dependências.

## Modelos de Chat

Neste curso, usaremos [Modelos de Chat](https://python.langchain.com/v0.2/docs/concepts/#chat-models), que fazem algumas coisas como receber uma sequência de mensagens como entrada e retornar mensagens de chat como saída. A LangChain não hospeda nenhum Modelo de Chat, em vez disso, contamos com integrações de terceiros. [Aqui](https://python.langchain.com/v0.2/docs/integrations/chat/) está uma lista de integrações de modelos de chat de terceiros dentro da LangChain! Por padrão, o curso usará [ChatOpenAI](https://python.langchain.com/v0.2/docs/integrations/chat/openai/) porque é popular e tem bom desempenho. Como mencionado, certifique-se de ter uma `OPENAI_API_KEY`.
Em alguns casos, usaremos Claude da Anthropic, que é uma alternativa ao GPT-4o! 



In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langchain_community tavily-python


In [4]:
import os


In [6]:
#Recebe a chave da API do OpenAI do ambiente criado (informado no README)
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# Define a variável de ambiente OPENAI_API_KEY se não estiver definida
_set_env("OPENAI_API_KEY")

# Agora, obtém a chave da API
openai_api_key = os.getenv("OPENAI_API_KEY", "not_found")
print(f"OpenAI API Key: {'*' * len(openai_api_key) if openai_api_key != 'not_found' else 'not_found'}")


OpenAI API Key: ********************************************************************************************************************************************************************


[Aqui](https://python.langchain.com/v0.2/docs/how_to/#chat-models) está um guia útil para todas as coisas que você pode fazer com modelos de chat, mas mostraremos alguns destaques abaixo. Se você executou `pip install -r requirements.txt` conforme indicado no README, então você instalou o pacote `langchain-openai`.

 Com isso, podemos instanciar nosso objeto de modelo `ChatOpenAI`. Se você está se inscrevendo na API pela primeira vez, deve receber [créditos gratuitos](https://community.openai.com/t/understanding-api-limits-and-free-tier/498517) que podem ser aplicados a qualquer um dos modelos. Você pode ver os preços para vários modelos [aqui](https://openai.com/api/pricing/). Os notebooks usarão por padrão o `gpt-4o-mini` porque é o mais em conta, e funciona bem!

Existem [alguns parâmetros padrão](https://python.langchain.com/v0.2/docs/concepts/#chat-models) que podemos definir com modelos de chat. Dois dos mais comuns são:

* `model`: o nome do modelo
* `temperature`: a temperatura de amostragem

A `Temperature` controla a aleatoriedade ou criatividade da saída do modelo, onde uma temperatura baixa (próxima a 0) produz saídas mais determinísticas e focadas. Isso é bom para tarefas que requerem precisão ou respostas factuais. Uma temperatura alta (próxima a 1) é boa para tarefas criativas ou para gerar respostas variadas.

In [7]:
#chamando o mini gpt-4o e o gpt-4o
from langchain_openai import ChatOpenAI
gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0)
gpt4o_mini_chat = ChatOpenAI(model="gpt-4o-mini", temperature=0)

Os modelos de chat na LangChain têm vários [métodos padrão](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface). Na maior parte do tempo, usaremos:

* `stream`: transmite de volta partes da resposta
* `invoke`: chama a cadeia em uma entrada

E, como mencionado, os modelos de chat recebem [mensagens](https://python.langchain.com/v0.2/docs/concepts/#messages) como entrada. As mensagens têm um papel (que descreve quem está dizendo a mensagem) e uma propriedade de conteúdo. Falaremos muito mais sobre isso depois, mas aqui vamos apenas mostrar o básico.

In [8]:
from langchain_core.messages import HumanMessage

#Cria uma mensagem
msg = HumanMessage(content="Como vai você?", name="Anderson")

# Lista de mensagens
messages = [msg]

#  funcao Invoke para passar a lista de mensagens
gpt4o_chat.invoke(messages)

AIMessage(content='Estou bem, obrigado por perguntar! Como posso ajudar você hoje?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 14, 'total_tokens': 27, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_a5d11b2ef2', 'finish_reason': 'stop', 'logprobs': None}, id='run-639e896a-abbe-487b-8f4d-31f551a3a6bb-0', usage_metadata={'input_tokens': 14, 'output_tokens': 13, 'total_tokens': 27})

We get an `AIMessage` response. Also, note that we can just invoke a chat model with a string. When a string is passed in as input, it is converted to a `HumanMessage` and then passed to the underlying model.


In [9]:
gpt4o_chat.invoke("Como vai você?")

AIMessage(content='Estou bem, obrigado por perguntar! Como posso ajudar você hoje?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 11, 'total_tokens': 24, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_a5d11b2ef2', 'finish_reason': 'stop', 'logprobs': None}, id='run-dd764eff-e6b7-4073-926c-9972469107fd-0', usage_metadata={'input_tokens': 11, 'output_tokens': 13, 'total_tokens': 24})

In [10]:
gpt4o_mini_chat.invoke("Como vai você?")

AIMessage(content='Estou bem, obrigado! E você, como está?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 11, 'total_tokens': 22, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'stop', 'logprobs': None}, id='run-a8310559-fd4b-4a98-84c2-1cef8895d387-0', usage_metadata={'input_tokens': 11, 'output_tokens': 11, 'total_tokens': 22})

## Ferramentas de Busca

Você também verá Tavily no README, que é um mecanismo de busca otimizado para LLMs e RAG, visando resultados de pesquisa eficientes, rápidos e persistentes. Como mencionado, é fácil se cadastrar e oferece um generoso nível gratuito. Algumas lições (no Módulo 4) usarão o Tavily por padrão, mas, é claro, outras ferramentas de busca podem ser usadas se você quiser modificar o código por conta própria.
Temos tido relativo sucesso usando DuckDuckGo, mas você pode usar qualquer outra ferramenta de busca que você gostar!

In [13]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# Define a variável de ambiente TAVILY_API_KEY se não estiver definida
_set_env("TAVILY_API_KEY")

# Agora, obtém a chave da API
tavily_api_key = os.getenv("TAVILY_API_KEY", "not_found")
print(f"Tavily API Key: {'*' * len(tavily_api_key) if tavily_api_key != 'not_found' else 'not_found'}")

Tavily API Key: *************************************


In [20]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily_search = TavilySearchResults(max_results=2)
search_docs = tavily_search.invoke("O que é a Scoras Digital Tecnologia Ltda? Responda em português do Brasil, apenas")

In [21]:
search_docs

[{'url': 'https://www.econodata.com.br/consulta-empresa/41805775000111-SCORAS-TECNOLOGIA-DIGITAL-LTDA',
  'content': 'A empresa SCORAS TECNOLOGIA DIGITAL tem CNPJ 41805775000111 e sede em Barueri, SP. Sua atividade principal é Consultoria em tecnologia da informação de acordo com o CNAE de código J-6204-0/00.'},
 {'url': 'https://scoras.com.br/',
  'content': 'Home - ScoraS Digital. Soluções Inteligentes, Resultados Extraordinários. Inovando com Tecnologia e Inteligência Artificial! Somos uma empresa apaixonada por desafios tecnológicos e especialistas em transformar. ideias em realidade por meio de soluções inovadoras.'}]

In [28]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily_search = TavilySearchResults(max_results=2)
search_docs = tavily_search.invoke("O que é a Scorasbot?")

In [29]:
search_docs

[{'url': 'https://scorasbot.com/',
  'content': 'ScorasBot, a company created to help other businesses improve and automate their processes is becoming an essential enterprise at the forefront of digital transformation.'},
 {'url': 'https://scorasbot.com/home/about-us/',
  'content': 'ScorasBot, a company created to help your business improve and automate its processes is becoming an essential enterprise at the forefront of digital transformation. Innovation Leader. Artificial Intelligence. Business Efficiency. Years Experience. 0 + Continuous Adaptation. Customer Focus.'}]